In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10


In [ ]:
# Substitua por sua própria imagem de prateleira
img = cv2.imread("img_prateleira.jpg")
img = cv2.resize(img, (256, 256))
blurred = cv2.GaussianBlur(img, (5, 5), 0)
ycrcb = cv2.cvtColor(blurred, cv2.COLOR_BGR2YCrCb)
ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
equalized = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

# Visualização
plt.imshow(cv2.cvtColor(equalized, cv2.COLOR_BGR2RGB))
plt.title("Imagem Equalizada")
plt.axis('off')
plt.show()


In [ ]:
gray = cv2.cvtColor(equalized, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
edges = cv2.Canny(thresh, 100, 200)

plt.imshow(edges, cmap='gray')
plt.title("Detecção de Bordas")
plt.axis('off')
plt.show()


In [ ]:
orb = cv2.ORB_create()
kp, des = orb.detectAndCompute(gray, None)
img_kp = cv2.drawKeypoints(equalized, kp, None, color=(0,255,0))

plt.imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
plt.title("Pontos-chave (ORB)")
plt.axis('off')
plt.show()


In [ ]:
M = cv2.getRotationMatrix2D((128, 128), 30, 1.0)
rotated = cv2.warpAffine(equalized, M, (256, 256))

plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
plt.title("Imagem Rotacionada")
plt.axis('off')
plt.show()


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=3, validation_data=(x_test, y_test_cat))


In [ ]:
loss, acc = model.evaluate(x_test, y_test_cat)
print(f"Acurácia: {acc*100:.2f}%")

preds = model.predict(x_test[:100])
y_pred = np.argmax(preds, axis=1)
y_true = y_test[:100].flatten()

print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))
